In [1]:
from tqdm import tqdm
from collections import defaultdict
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
import time
from collections import deque
from threading import Lock
import asyncio

In [2]:
# Copy RePASs repo to validate our results - ONLY RUN ONCE
#!git clone https://github.com/RegNLP/RePASs.git && cd RePASs

In [3]:
# Cargar variables de entorno
load_dotenv()

False

In [4]:
# Proceso para cargar una colección de pasajes desde el directorio de documentos estructurados
ndocs = 40  # Número de documentos a procesar
passages = defaultdict(str) # Lista para almacenar la colección de pasajes

# Lee cada documento y extrae los pasajes relevantes
for i in range(1, ndocs + 1):
    with open(os.path.join("ObliQADataset/StructuredRegulatoryDocuments", f"{i}.json")) as f:
        doc = json.load(f)  # Carga el contenido del documento JSON
        for psg in doc:  # Recorre cada pasaje del documento
            passages[psg["ID"]] = psg["Passage"]

In [5]:
rankings_dict = defaultdict(list)

# Abrimos el archivo de rankings y lo cargamos en memoria
with open('data/rankings_hybrid.trec', 'r') as f:
    # Formato de línea: QuestionID Q0 DocumentID Rank Score Método
    for line in f:
        parts = line.strip().split()
        question_id = parts[0]
        document_id = parts[2]
        rank = int(parts[3])
        score = float(parts[4])
        rankings_dict[question_id].append({
            'doc': document_id,
            'score': score
        })

## Standard deployment

Standard deployment using gpt 3.5 turbo

In [6]:
endpoint = os.getenv('QNA_ENDPOINT_URL')
openAIKey = os.getenv('QNA_OPENAI_API_KEY')
llm_model = 'gpt-35-turbo'

if not endpoint:
    raise ValueError("No se ha definido la variable de entorno QNA_ENDPOINT_URL")

if not openAIKey:
    raise ValueError("No se ha definido la variable de entorno QNA_OPENAI_API_KEY")

openAI_client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=openAIKey,
    api_version="2024-05-01-preview"
)

# Esta clase limita el numero de veces que se puede llamar una funcion en un intervalo de tiempo
# dado. Garantiza que las funcion se llame todas las veces pero no el mismo orden que fueron invocadas
class Throttle:
    def __init__(self, rate_limit, time_window):
        self.rate_limit = rate_limit # Numero de llamadas permitidas por fraccion de tiempo
        self.time_window = time_window # Fraccion de tiempo en segundos
        self.calls = deque() # Almacena los tiempos de las llamadas en un deque (Array que permite agregar y remover elementos de ambos extremos en O(1))
        self.lock = Lock() # Lock para asegurar que solo un hilo accede a la lista de llamadas a la vez
        self.queue = asyncio.Queue() # Una cola para almacenar las llamadas que se hicieron mientras se estaba esperando e intentar nuevamente

    def __call__(self, func):
        async def wrapped_func(*args, **kwargs):
            # Referenciar la funcion fuera del context local
            nonlocal func
            # Bloquear el acceso a la lista de llamadas
            with self.lock:
                current_time = time.time()
                
                # Remover llamadas que ya no están en la ventana de tiempo
                while self.calls and self.calls[0] < current_time - self.time_window:
                    self.calls.popleft()

                # Si todavia hay espacio en la ventana de tiempo, agregar la llamada actual y ejecutar la función    
                if len(self.calls) < self.rate_limit:
                    self.calls.append(current_time)
                    return await func(*args, **kwargs)
                else:
                    # De lo contrario, agregar la llamada a la cola y esperar
                    await self.queue.put((func, args, kwargs)) # Agregar la llamada a la cola
                    
                    # Procesa las llamadas en la cola
                    while not self.queue.empty():
                        # Sacar la llamada de la cola
                        func, args, kwargs = await self.queue.get()
                        current_time = time.time()

                        # Remover llamadas que ya no están en la ventana de tiempo
                        while self.calls and self.calls[0] < current_time - self.time_window:
                            self.calls.popleft()

                        # Si ya hay espacio en la ventana de tiempo, ejecutar la función y sacar de la cola
                        if len(self.calls) < self.rate_limit:
                            self.calls.append(current_time)
                            result = await func(*args, **kwargs)
                            self.queue.task_done()
                            return result
                        else:
                            # De lo contrario, esperar 10 segundos antes de intentar nuevamente
                            await self.queue.put((func, args, kwargs)) # Re-ingresar llamada a la fila
                            await asyncio.sleep(min(10, self.time_window))
                    
        return wrapped_func
    
def build_prompt(question: str, relevant_passages: list[str]):
    """
    Utilizar un modelo LLM para crear una respuesta a la pregunta a partir de la lista
    de pasajes relevantes.
    """

    # Construir el prompt para el modelo LLM
    system_prompt = """You are a regulatory compliance assistant. Provide a **complete**, **coherent**, and **correct** response to the given question by synthesizing the information from the provided passages. Your answer should **fully integrate all relevant obligations, best practices, and insights**, and directly address the question. The passages are presented in order of relevance, so **prioritize the information accordingly** and ensure consistency in your response, avoiding any contradictions. Additionally, reference **specific regulations and key compliance requirements** outlined in the regulatory content to support your answer. **Do not use any extraneous or external knowledge** outside of the provided passages when crafting your response.
    """

    user_prompt = f"Question: {question}\n\nPassages:\n\n"
    for idx, passage in enumerate(relevant_passages, 1):
        user_prompt += f"{passage}\n\n"
        
    return (system_prompt, user_prompt)

# Permite maximo 60 llamadas cada 70 segundos. Las llamadas se van encolando y se ejecutaran a medida que se libere espacio
@Throttle(rate_limit=60, time_window=70)
async def summarize_answer(question: str, relevant_passages: list[str]) -> str:
    """
    Utilizar un modelo LLM para crear una respuesta a la pregunta a partir de la lista
    de pasajes relevantes.
    """

    (system_prompt, user_prompt) = build_prompt(question, relevant_passages)

    completion = openAI_client.chat.completions.create(
        model=llm_model, # Estamos usando el modelo gpt-3.5-turbo
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.25, # Controla la aleatoriedad de las respuestas generadas - Menor valor, respuestas más deterministicas
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=None,
        stream=False,
        max_tokens=800,
    )

    return completion.choices[0].message.content
    

In [7]:
def extract_passages(questionId: str) -> list[str]:
    """
    Extrae los pasajes para la pregunta dada relevantes para el modelo LLM
    """
    retrieved_passages = []
    should_stop = False
    
    for i in range(len(rankings_dict[questionId])):
        # Si hubo una diferencia en relevancia significativa entre dos pasajes, no extraer mas pasajes
        # Si ya se extrajeron 10 pasajes, no extraer mas
        if should_stop or len(retrieved_passages) == 10:
            break
            
        # Si no se ha extraido ningun pasaje, extraer al menos uno
        if len(retrieved_passages) == 0:
            retrieved_passages.append(rankings_dict[questionId][i]["doc"])
            continue
                
        # Revisar si hay una diferencia en relevancia entre este y el siguiente pasaje de mas del 10%
        if i < len(rankings_dict[questionId]) - 1 and rankings_dict[questionId][i]["score"] - rankings_dict[questionId][i+1]["score"] > 0.1:
                should_stop = True

        # No incluir pasajes poco relevantes
        if rankings_dict[questionId][i]["score"] < 0.72:
            break

        retrieved_passages.append(rankings_dict[questionId][i]["doc"])
        
    # Extraer el texto plano
    retrieved_passages = [passages[doc] for doc in retrieved_passages]
    
    return retrieved_passages

In [10]:
answers = []

# Abrimos el archivo JSON que contiene las consultas de prueba (ObliQA_test.json)
with open("ObliQADataset/ObliQA_test.json") as f:
    data = json.load(f)  # Cargamos el contenido del archivo JSON
    
    # Iteramos sobre cada entrada (pregunta) en el archivo de datos
    for e in tqdm(data):  # tqdm agrega una barra de progreso durante la iteración
        query = e['Question']  # Extraemos la pregunta o consulta desde el campo 'Question'
        question_id = e["QuestionID"] # Extraemos el ID de la pregunta
        
        retrieved_passages = extract_passages(question_id)

        answer = await summarize_answer(query, retrieved_passages)

        answers.append({
            "QuestionID": question_id,
            "RetrievedPassages": retrieved_passages,
            "Answer": answer
        })

# Guardamos las respuestas en un archivo JSON
with open("data/answers.json", "w") as f:
    json.dump(answers, f, indent=2)        

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2786/2786 [1:56:08<00:00,  2.50s/it]


## Batch deployment

Batch deployment using gpt-4o-mini

In [21]:
def queue_batch_summarization_job(jobs):
    """
    Crea un job en Azure con todas las preguntas usando GPT-4o-Mini
    """
    endpoint = os.getenv('QNA_ENDPOINT_URL')
    openAIKey = os.getenv('QNA_OPENAI_API_KEY')

    if not endpoint:
        raise ValueError("No se ha definido la variable de entorno QNA_ENDPOINT_URL")

    if not openAIKey:
        raise ValueError("No se ha definido la variable de entorno QNA_OPENAI_API_KEY")

    openAI_client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=openAIKey,
        api_version="2024-08-01-preview"
    )

    file_name = "data/batch_questions.jsonl"

    with open(file_name, 'w') as file:
        for job in jobs:
            file.write(json.dumps(job) + '\n')

    batch_file = openAI_client.files.create(
      file=open(file_name, "rb"),
      purpose="batch"
    )
    
    while True:
        file = openAI_client.files.retrieve(batch_file.id)
        if file.status == "processed" or file.status == "error":
            break
        time.sleep(10)
    
    batch_job = openAI_client.batches.create(
      input_file_id=batch_file.id,
      endpoint="/v1/chat/completions",
      completion_window="24h"
    )
    
    return batch_job

In [ ]:
jobs = []

# Abrimos el archivo JSON que contiene las consultas de prueba (ObliQA_test.json)
with open("ObliQADataset/ObliQA_test.json") as f:
    data = json.load(f)  # Cargamos el contenido del archivo JSON
    
    # Iteramos sobre cada entrada (pregunta) en el archivo de datos
    for e in tqdm(data):  # tqdm agrega una barra de progreso durante la iteración
        query = e['Question']  # Extraemos la pregunta o consulta desde el campo 'Question'
        question_id = e["QuestionID"] # Extraemos el ID de la pregunta

        retrieved_passages = extract_passages(question_id)

        (system_prompt, user_prompt) = build_prompt(query, retrieved_passages)
        
        jobs.append({
            "custom_id": question_id,
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "gpt-4o-mini",
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                "temperature": 0.25,
                "frequency_penalty": 0.0,
                "presence_penalty": 0.0,
                "max_tokens": 800,
            }
        })
        
batch_job = queue_batch_summarization_job(jobs)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2786/2786 [00:00<00:00, 91025.68it/s]


In [ ]:
# Leer los resultados una vez el job haya finalizado y 
# cargar el archivo resultante para procesarlo segun el formato requerido


In [ ]:
## Script para evaluar los resultados que se guardaran en /RePASs/data/hybrid
## Se deben correr activando el ambiente virtual definido en RePASs

#python scripts/evaluate_model.py --input_file ./../data/answers.json --group_method_name hybrid